In [140]:
import regex as re
from functional import seq
from functional.pipeline import Sequence
from fn import _
from collections import namedtuple
from regex.regex import Match, Pattern
from typing import List, Dict, Tuple, Optional
from termcolor import colored
import os


In [88]:
#Nr doesnt matter, position does

In [89]:
act_dir = os.environ["HOME"]+"/tmp/nlp/ustawy"
#act_file = "1993_599.txt"

In [141]:
# act = open(act_dir + "/2003_592.txt").read() [20265:] # just footers
# act = open(act_dir + "/2003_592.txt").read() [:20265] # no footers
act = open(act_dir + "/2003_592.txt").read() #all
print(act)




Tekst ustawy
ustalony ostatecznie po rozpatrzeniu poprawek Senatu
 
 
 
USTAWA
z dnia 5 września
2008 r.
 
o komercjalizacji
państwowego przedsiębiorstwa użyteczności publicznej "Poczta Polska" 
 

Rozdział 1 
Przepisy ogólne 
 
Art. 1.
Ustawa reguluje
zasady i tryb komercjalizacji państwowego przedsiębiorstwa użyteczności
publicznej "Poczta Polska" działającego na podstawie ustawy z dnia 30 lipca
1997 r. o państwowym przedsiębiorstwie użyteczności publicznej "Poczta Polska"
(Dz. U. Nr 106, poz. 675, z późn. zm.[1])),
zwanego dalej "Pocztą Polską". 
 
Art. 2.
1.  Komercjalizacja
Poczty Polskiej, w rozumieniu ustawy, polega na przekształceniu Poczty Polskiej
w spółkę akcyjną o nazwie "Poczta Polska Spółka Akcyjna", zwaną dalej "Spółką",
w której Skarb Państwa jest jedynym akcjonariuszem. 
2.  Spółka
wstępuje we wszystkie stosunki prawne, których podmiotem była Poczta Polska,
bez względu na charakter prawny tych stosunków, jeżeli przepisy ustawy nie
stanowią inaczej. 
 
Art. 3.
Spółce

In [142]:
def match_with_index(pattern ,text:str)-> List[Tuple[int,int,str]]:
    return [(m.start(), m.end(),m.captures()[0]) for m in re.finditer(pattern, text)]


def print_highlighted(
    text:str,
    matches_groups:List[List[Tuple[int,int]]],
    colors:List[Tuple[str,Optional[str]]]
):
    if len(matches_groups) != len(colors):
        raise "There should be the same number of matches groups and colors"
#     if seq(matches_groups).exists(lambda x: len(x) == 0):
#         print("sory bro, cannot do this - will loop")
#         return
    
    
    MatchedEntry=namedtuple('MatchedEntry','beg end color on_color')
    
    # Seq[MatchedEntry]
    mg = seq(matches_groups)\
        .zip(colors)\
        .flat_map(lambda gc: seq(gc[0])\
            .map(lambda x : MatchedEntry(beg=x[0],end=x[1],color=gc[1][0], on_color = gc[1][1]))
                 )\
        .order_by(_.beg)
        
    # Will duplicate some matches but that's not the point
    beg = 0
    for matched in mg:
        print(text[beg:matched.beg],end="") 
        print(colored(text[matched.beg:matched.end],color= matched.color, on_color= matched.on_color),end="")
        beg = matched.end
    print(text[beg:])

In [143]:
dzu_pat = re.compile("Dz\.?\s?U.?")
year_pat = re.compile("(?P<year>((19\d)|(200)|(201))\d{1})")
nr_pat = re.compile("Nr\s+(?P<nr>\d+)") #TODO: word sep or sth
pos_pat = re.compile("poz\.\s+(?P<poz>\d+(-\d+)*)") #todo - "i \d"

#Entry extractors
footer_ext_pat = re.compile("\[\d+\]\)?\s+Zmiany(.|\s|\n)*?w\s+Dz\.*\s*U\.?(?P<foot_entry>(.|\s|\n)*?)(?=(\[|\Z))")
art_ext_pat=re.compile("(U|u)staw\w*\s+(z\s+dnia\s+)+\d+\s+\w+\s+(?P<year>\d{4})(\s|\n)*r\.?\s*(–|\-)*\s*(?P<title>(\w|\s|\")*)\((?P<art_entry>(.|\n)*?)\)")

In [144]:
ExtRef= namedtuple("ExtRef", "year nr poz title")

def extract_group_name(match:Match) -> str:
    return list(match.groupdict().keys())[0]

#Assumption: entry without year  takes year and title from outside
def art_entry_extractor(title:str,default_year:str,entry:str)-> List[ExtRef]:
    matches = seq([year_pat,nr_pat,pos_pat])\
    .flat_map(lambda pat: list(pat.finditer(entry)))\
    .order_by(lambda x: x.start())
    
    year_found = False
    year = default_year
    nr = None
    res = []    
    for match in matches:
        name = extract_group_name(match)
        if name == "year":
            year_found = True
            year = match.groupdict()[name]
            if type(year) != str:
                raise "Oh no, riots in your code dear developer. Year is supposed to be str."
        elif name =="nr":
            nr = match.groupdict()[name]
            if type(nr) != str:
                raise "Oh no, riots in your code dear developer. Nr is supposed to be str."
        elif name =="poz":
            poz = match.groupdict()[name]
            if type(poz) != str:
                raise "Oh no, riots in your code dear developer. poz is supposed to be str."
            res.append(ExtRef(year=year,nr=nr,poz=poz, title=None if year_found else title))
        else:
            print("Disaster here; Match name not found ion art_entry_extractor")
    return res


def art_match_extractor(match:Match)-> List[ExtRef]:
    d = match.groupdict()
    title = re.sub("[\s\n]+"," ",d["title"]).strip()
    return art_entry_extractor(title,d["year"],d["art_entry"])

    
def foot_match_extractor(match:Match)-> List[ExtRef]:
    return art_entry_extractor(None,None, match.groupdict()["foot_entry"])

In [192]:
def ext_ref_extractor(text:str) -> Sequence: #Sequence[ExtRef]:
    art_matches = seq(list(art_ext_pat.finditer(text))).flat_map(art_match_extractor)
    footer_matches = seq(list(footer_ext_pat.finditer(text))).flat_map(foot_match_extractor)
    return  art_matches + footer_matches



CountedExtRef= namedtuple("CountedExtRef", "count year nr poz title")
def ext_ref_counter(refs:Sequence)-> CountedExtRef:#Sequence[ExtRef] -> CountedExternalRef
    def counted_from_ext_ref(count:int,ref:ExtRef) -> CountedExtRef:
        return CountedExtRef(count=count,year=ref.year,nr=ref.nr,poz=ref.poz,title=ref.title)
    
    def count_and_select_best(t:Tuple[str,Sequence])-> CountedExtRef:
        if len(t[1]) == 0:
            raise "Aggregated tuple shouldn't be empty"
        with_title = seq(t[1]).find(lambda ref:ref.title != None)
        return counted_from_ext_ref(len(t[1]), t[1][0] if with_title is None else with_title)
        
        
    return refs.group_by(lambda ref : ref.year+ "-"+ref.poz).map(count_and_select_best)



def display_seq(sequence:Sequence,rows:int)-> None:
    sequence._repr_html_= lambda :sequence.tabulate(rows,tablefmt='html')
    display(sequence)
    sequence._repr_html_= lambda :sequence.tabulate(10,tablefmt='html')

    

    
res = ext_ref_counter(ext_ref_extractor(act))\
    .order_by(lambda x : int(x.poz))\
    .order_by(lambda x : int(x.year))\
    .order_by(lambda x: 1/x.count)
display_seq(
    res,
    50
)

count,year,nr,poz,title
7,2006,104,708,
7,2006,157,1119,o nadzorze nad rynkiem finansowym
5,2004,96,959,
5,2005,183,1538,
4,2003,60,535,Prawo upadłościowe i naprawcze
4,2005,184,1539,
3,2002,240,2055,
3,2003,130,1188,Prawo pocztowe
3,2004,146,1546,
3,2004,173,1808,


In [158]:
ala = namedtuple("ala", "a l")
display(seq([ala(a=1,l=3),ala(a=2,l=4)]))

a,l
1,3
2,4


In [134]:
matches = seq([
    #year_pat,
    dzu_pat,
    nr_pat,
    pos_pat,
    ustaw_pat
#     footer_pat
]).map(lambda pattern: match_with_index(pattern,act)).to_list()


print_highlighted(act,matches
                  ,[
                      #("green","on_grey"),
                      ("red","on_grey"),
                      ("blue","on_grey"),
                      ("yellow","on_grey"),
                      ("grey","on_yellow"),
                  ])




Tekst ustawy
ustalony ostatecznie po rozpatrzeniu poprawek Senatu
 
 
 
USTAWA
z dnia 5 września
2008 r.
 
o komercjalizacji
państwowego przedsiębiorstwa użyteczności publicznej "Poczta Polska" 
 

Rozdział 1 
Przepisy ogólne 
 
Art. 1.
Ustawa reguluje
zasady i tryb komercjalizacji państwowego przedsiębiorstwa użyteczności
publicznej "Poczta Polska" działającego na podstawie ustawy z dnia 30 lipca
1997 r. o państwowym przedsiębiorstwie użyteczności publicznej "Poczta Polska"
(Dz. U. Nr 106, poz. 675, z późn. zm.[1])Dz. U. Nr 106, poz. 675, z późn. zm.[1])),
zwanego dalej "Pocztą Polską". 
 
Art. 2.
1.  Komercjalizacja
Poczty Polskiej, w rozumieniu ustawy, polega na przekształceniu Poczty Polskiej
w spółkę akcyjną o nazwie "Poczta Polska Spółka Akcyjna", zwaną dalej "Spółką",
w której Skarb Państwa jest jedynym akcjonariuszem. 
2.  Spółka
wstępuje we wszystkie stosunki prawne, których podmiotem była Poczta Polska,
bez względu na charakter prawny tych stosunków, jeżeli przepisy ustawy

Find all external references to bills, e.g. **ustawie z dnia 4 marca 1994 r. o zakładowym funduszu świadczeń socjalnych (Dz. U. z 2012 r. poz. 592)**. The result should be aggregated by bill ID (year and position) and sorted by descending number of reference counts. The reference format should include:
        the title of the regulation (if present)
        the year of the regulation
        the number of the Journal of Laws of the Republic of Poland (Dziennik Ustaw) - if applicable
        the position of the regulation


In [ ]:
match_with_index(footer_pat,act)